In [7]:
import pandas as pd
import re
import numpy as np

In [8]:
data_location = "data/properatti.csv" # dataset en carpeta data
data = pd.read_csv(data_location)
data_inicial = data.copy()
data.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [9]:
# Salteo un monton de celdas descriptivas

#Aplicamos regex para obtener los precios desde la descripción
patron_price = "(?P<Currency>U\$S|USD|uSd|usD|usd|Usd|US\$|us\$|uS\$|Us\$)\$? ?(?P<Price>\d+(\,|\.)?\d+((\,|\.)\d+)?)"
regex_price = re.compile(patron_price, flags = re.IGNORECASE)
data_search = data['description'].apply(lambda x: x if x is np.NaN else regex_price.search(x))
data_search_not_null = data_search.notnull()
data.loc[data_search_not_null,'description_price'] = data_search[data_search_not_null].apply(lambda x : x.group('Price'))
data.loc[data_search_not_null,'description_currency'] = data_search[data_search_not_null].apply(lambda x : x.group('Currency'))

In [10]:
#Aplicamos regex para obtener los ambientes desde la descripción
patron_rooms = "(?P<rooms>\d|Mono|mono|dos|Dos|DOS|tres|Tres|TRES|cuatro|Cuatro|CUATRO) ?(AMB|amb)"
regex_rooms = re.compile(patron_rooms, flags = re.IGNORECASE)
data_search = data['description'].apply(lambda x: x if x is np.NaN else regex_rooms.search(x))
data_search_not_null = data_search.notnull()
data.loc[data_search_not_null,'description_rooms'] = data_search[data_search_not_null].apply(lambda x : x.group('rooms'))
data['description_rooms'] = data['description_rooms'].replace(['MONO', 'Mono', 'mono'], 1) 
data['description_rooms'] = data['description_rooms'].replace(['dos','Dos', 'DOS'], 2) 
data['description_rooms'] = data['description_rooms'].replace(['tres','Tres','TRES'], 3) 
data['description_rooms'] = data['description_rooms'].replace(['cuatro','Cuatro', 'CUATRO'], 4) 

In [11]:
#Creamos la columna con los ambientes obtenidos en regex
data['rooms_clean'] = data['rooms']
mask_rooms_null = data["rooms"].isnull()
data.loc[mask_rooms_null, "rooms_clean"] = data.loc[mask_rooms_null, "description_rooms"]

#Observamos reducción de nulls
print (data["rooms"].isnull().sum())
print (data["rooms_clean"].isnull().sum())
#Reemplazamos el resto con "No informa"
data.rooms_clean.fillna('0', inplace=True)
data["rooms_clean"] = data.rooms_clean.astype(int)
data.rooms_clean.replace(0, 'No informa', inplace=True)
data.rooms_clean.value_counts()

73830
48592


No informa    48618
2             19961
3             19507
1             12743
4             11758
5              4862
6              1729
7               891
8               425
10              226
9               198
11               72
12               65
13               33
14               27
15               26
17               16
16               11
20               11
22                8
18                6
21                5
19                4
25                4
30                4
32                3
23                2
28                1
24                1
27                1
31                1
29                1
Name: rooms_clean, dtype: int64

In [12]:
#Se fueron muestreando y analizando las descripciones de los outliers, suenan razonables
#ya que se venden propiedades con varios departamentos o edificios pero también hay casos
#donde se observa una incorreción en la carga por lo tanto al ser tan pocos casos se elige clasificarlos como "No informa" 

data_copy = data.copy()
data_copy.rooms_clean.replace('No informa', 0, inplace=True)
#print(* data_copy["description"][data_copy["rooms_clean"]==25], sep="\n\n")
print ( data_copy.rooms_clean.value_counts())

data.loc[data_copy["rooms_clean"]>11, "rooms_clean"] = 'No informa'

print ( data.rooms_clean.value_counts())

0     48618
2     19961
3     19507
1     12743
4     11758
5      4862
6      1729
7       891
8       425
10      226
9       198
11       72
12       65
13       33
14       27
15       26
17       16
16       11
20       11
22        8
18        6
21        5
19        4
25        4
30        4
32        3
23        2
31        1
24        1
27        1
28        1
29        1
Name: rooms_clean, dtype: int64
No informa    48848
2             19961
3             19507
1             12743
4             11758
5              4862
6              1729
7               891
8               425
10              226
9               198
11               72
Name: rooms_clean, dtype: int64


In [13]:
#Aplicamos regex para obtener los pisos desde la descripción
patron_floor = "(?P<floor>\d+|primer|Primer|PRIMER|primero|Primero|PRIMERO|segunda|Segunda|SEGUNDA|segundo|Segundo|SEGUNDO|tercer|Tercer|TERCER|cuarto|Cuarto|CUARTO|quinto|Quinto|QUINTO|sexto|Sexto|SEXTO|septimo|Septimo|SEPTIMO|octavo|Octavo|OCTAVO) ?(PISO|piso|PLANTA|planta)"
regex_floor = re.compile(patron_floor, flags = re.IGNORECASE)
data_search = data['description'].apply(lambda x: x if x is np.NaN else regex_floor.search(x))
data_search_not_null = data_search.notnull()
data.loc[data_search_not_null,'description_floors'] = data_search[data_search_not_null].apply(lambda x : x.group('floor'))
data['description_floors'] = data['description_floors'].replace(['primero',
                                                                 'Primero',
                                                                 'PRIMERO',
                                                                 'primer',
                                                                 'Primer',
                                                                 'PRIMER'], 1) 
data['description_floors'] = data['description_floors'].replace(['segundo',
                                                                 'Segundo',
                                                                 'SEGUNDO',
                                                                 'segunda',
                                                                 'Segunda',
                                                                 'SEGUNDA'], 2) 
data['description_floors'] = data['description_floors'].replace(['tercer',
                                                                 'Tercer',
                                                                 'TERCER',
                                                                 'TErcer'], 3) 
data['description_floors'] = data['description_floors'].replace(['cuarto',
                                                                 'Cuarto',
                                                                 'CUARTO'], 4) 
data['description_floors'] = data['description_floors'].replace(['quinto',
                                                                 'Quinto',
                                                                 'QUINTO'], 5) 
data['description_floors'] = data['description_floors'].replace(['sexto',
                                                                 'Sexto',
                                                                 'SEXTO'], 6) 
data['description_floors'] = data['description_floors'].replace(['septimo',
                                                                 'SEPTIMO',
                                                                 'Septimo'], 7) 
data['description_floors'] = data['description_floors'].replace(['octavo',
                                                                 'Octavo',
                                                                 'OCTAVO'], 8) 
data['description_floors'] = data['description_floors'].replace('', 0) 

In [14]:
#Creamos la columna con los floor obtenidos en regex
data['floor_clean'] = data['floor']
mask_floor_null = data["floor"].isnull()
data.loc[mask_floor_null, "floor_clean"] = data.loc[mask_floor_null, "description_floors"]

#Observamos reducción de nulls
print (data["floor"].isnull().sum())
print (data["floor_clean"].isnull().sum())
#Reemplazamos el resto con "No informa"
data.floor_clean.fillna('0', inplace=True)
data["floor_clean"] = data.floor_clean.astype(int)
data.floor_clean.replace(0, 'No informa', inplace=True)
data.floor_clean.value_counts()

113321
95918


No informa    95958
2              6560
1              5913
3              2660
4              1519
              ...  
433               1
410               1
407               1
406               1
306               1
Name: floor_clean, Length: 308, dtype: int64

In [15]:
#Dado que los edificios más altos en Argentina tienen 27 pisos (Fuente="https://es.wikipedia.org/wiki/Anexo:Edificios_m%C3%A1s_altos_de_Argentina")
#los que superen ese limite se clasifican como "No informa" 

data_copy = data.copy()
data_copy.floor_clean.replace('No informa', 0, inplace=True)
print ( data_copy.floor_clean.value_counts())

data.loc[data_copy["floor_clean"]>27, "floor_clean"] = 'No informa'

print ( data.floor_clean.value_counts())

0       95958
2        6560
1        5913
3        2660
4        1519
        ...  
1379        1
803         1
802         1
98          1
157         1
Name: floor_clean, Length: 308, dtype: int64
No informa    96665
2              6560
1              5913
3              2660
4              1519
8              1089
6              1054
5               965
10              858
7               811
9               794
12              543
14              496
11              282
13              265
15              164
18              104
24               84
17               72
16               59
20               59
19               45
22               43
21               41
26               25
27               21
25               19
23               10
Name: floor_clean, dtype: int64


In [16]:
#Dado el alto número de NA en expenses decidimos volverla una variable booleana que determina si tiene o no expensas.
data['with_expenses 1'] = data['expenses'].fillna(0)>0
data['with_expenses 2'] = data.description.str.contains('expensa')
data['with_expenses 3'] = data.description.str.contains('sin expensa|no expensa' , na=False)
data['with_expenses'] = ((data['with_expenses 1']) | ((~data['with_expenses 3']) & data['with_expenses 2']))
data['with_expenses']

0         False
1         False
2         False
3         False
4         False
          ...  
121215     True
121216     True
121217    False
121218    False
121219     True
Name: with_expenses, Length: 121220, dtype: bool

In [17]:
data_description_with_floor = data.description.str.contains('piso|planta' , na=False)
data.description[data_description_with_floor].shape[0]/data.description.shape[0]

0.4292031017983831

In [18]:
#Agregamos columnas que identifican la existencia de determinados amenities
data_description_with_pool = data.description.str.contains('pileta|piscina|alberca')
data ['pool'] = data_description_with_pool
data_description_with_Terrace = data.description.str.contains('terraza')
data ['terrace'] = data_description_with_Terrace
data_description_with_Balcony_or_yard  = data.description.str.contains('balcon|patio')
data ['balcony_or_yard'] = data_description_with_Balcony_or_yard
data_description_with_grill  = data.description.str.contains('parrilla|asador')
data ['grill'] = data_description_with_grill
data_description_with_underfloor_heating  = data.description.str.contains('radiante|calefacción central')
data ['underfloor_heating'] = data_description_with_underfloor_heating
data_garage  = data.description.str.contains('cochera|garage|garaje|estacionamiento|aparcamiento')
data['Garage'] = data_garage
amenities = ['Garage' ,
             'underfloor_heating' ,
             'grill' ,
             'balcony_or_yard' ,
             'terrace' ,
             'pool']

In [19]:
# Vamos con la parte geo

# Como todas las filas corresponden a Tigre le asigno ese falor
data['place_name'] = data['place_name'].fillna('Tigre')

In [20]:
# Quiero verificar que los datos en place_name coincidan con place_with_parent_names
data['place_name_parent'] = data.apply(lambda x: x['place_with_parent_names'].split('|')[-2], axis = 1)
mask_diff_place_name = data['place_name_parent'] != data['place_name']
data.loc[mask_diff_place_name, 'place_with_parent_names'].value_counts()

|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    23
Name: place_with_parent_names, dtype: int64

In [21]:
'''No se computaron 23 valores de 'place_name_parent' porque 'place_with_parent_names' tiene doble ||
en vez de un sólo |'''
data.loc[data[mask_diff_place_name].index, 'place_name_parent'] = 'Tigre'
# Ahora sí comparo y veo que no hay diferencia
(data['place_name'] != data['place_name_parent']).sum()

0

In [22]:
# Quiero separar los datos por provincia y eliminar aquellas que tengan pocos registros
data['province'] = data.apply(lambda x: x['place_with_parent_names'].split('|')[2], axis = 1)
province_group = data.groupby('province')
province_group['province'].count().sort_values(ascending = False)

province
Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
La 

In [23]:
# Elimino las provincias que tengan menos de 1000 registros
data_province_filtered = province_group.filter(lambda grp: grp['province'].count() > 1000)
data_province_filtered['province'].value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Name: province, dtype: int64

In [24]:
''' Se evalúa extraer información de la columna 'geonames_id', primero se verá
el porcentaje de columnas 'lat' que sean NaN y que tienen información en 'geonames_id'.'''
mask_lat_nan_and_geonames_notnan = data['lat'].isna() & data['geonames_id'].notna()

print('El porcentaje de lat NaNs y que contienen información en geonames_id es: %',
      round(mask_lat_nan_and_geonames_notnan.sum() * 100 / mask_lat_nan_and_geonames_notnan.shape[0], 2)
     )

El porcentaje de lat NaNs y que contienen información en geonames_id es: % 35.77


In [25]:
data_geonames = pd.read_csv('./data/AR.txt', header = None, sep = '\t')
data_geonames = data_geonames[[0, 1, 4, 5, 17]]
data_geonames = data_geonames.rename(columns = {0 :  'geonames_id',
                                                1 : 'localidad_API',
                                                4 : 'lat_API',
                                                5 : 'lon_API',
                                                17 : 'place_parent_names_API'})
data_geonames['place_name_API'] = data_geonames['place_parent_names_API'].apply(lambda x: 
                                x if x is np.nan else x[x.rfind('/') + 1:])
data_geonames['place_name_API'] = data_geonames['place_name_API'].apply(lambda x: 
                                x if x is np.nan else x.replace('_', ' '))
display(data_geonames.head(3))
print(data_geonames.shape)

,geonames_id,localidad_API,lat_API,lon_API,place_parent_names_API,place_name_API
0,3427200,Sierra del 15,-37.96613,-57.94670,America/Argentina/Buenos_Aires,Buenos Aires
1,3427201,Puesto 1 Napoleón Cué,-28.63333,-56.11667,America/Argentina/Cordoba,Cordoba
2,3427202,Cañada del Zorro,-32.16667,-59.50000,America/Argentina/Cordoba,Cordoba


(49673, 6)
